In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2022-08-25T19:11:04.183451-04:00

Python implementation: CPython
Python version       : 3.8.9
IPython version      : 8.4.0

Compiler    : Clang 13.1.6 (clang-1316.0.21.2.5)
OS          : Darwin
Release     : 21.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 20
Architecture: 64bit



In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import sklearn
import gc
import os

In [4]:
%watermark --iversions

pandas : 1.4.3
sklearn: 1.1.2
numpy  : 1.23.1



In [5]:
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

In [6]:
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X = train[train.columns[1:]].values/255
X_test = test[test.columns[1:]].values/255
Y = train.label.values
Y_test = test.label.values

In [9]:
%%time
pca = PCA(n_components=0.9, whiten=True)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)
X.shape

CPU times: user 47.5 s, sys: 1.58 s, total: 49.1 s
Wall time: 4.28 s


(60000, 84)

In [10]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=400, validation_fraction=None, 
                                           random_state=137, max_depth=15)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()


print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

Fitting fold 1
Fold accuracy: 0.89275
Fitting fold 2
Fold accuracy: 0.88725
Fitting fold 3
Fold accuracy: 0.8924166666666666
Fitting fold 4
Fold accuracy: 0.8870833333333333
Fitting fold 5
Fold accuracy: 0.8856666666666667
0.8890333333333333
0.8925
CPU times: user 2h 15min 15s, sys: 3h 8min 52s, total: 5h 24min 8s
Wall time: 44min 48s


In [25]:
0.9713333333333334

0.9713333333333334